In [439]:
import random
from hashlib import sha256

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

def schnorr_sign(message, g, p, s):
    # 选择随机数k
    k = random.randint(1, p-2)
    # 计算r = g^k mod p
    r = pow(g, k, p)
    # 将消息和r的值哈希，然后与私钥s结合产生签名
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    x = (k - s * hash_value) % (p-1)
    return r, x

def schnorr_verify(message, g, p, v, r, x):
    # 重新计算r的哈希值
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    # 使用公钥v和签名x、r计算验证值
    rv = (pow(g, x, p) * pow(v, hash_value, p)) % p
    return rv == r

# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

# 生成密钥
s, v = generate_keys(g, p)

# 签名消息
message = "Hello, Schnorr!"
r, x = schnorr_sign(message, g, p, s)
print(f"Signature: (r={r}, x={x})")

# 验证签名
valid = schnorr_verify(message, g, p, v, r, x)
print(f"Signature valid: {valid}")


Signature: (r=674137276, x=204078983)
Signature valid: True


In [440]:
# 设置大素数p和其原根g
p = 303864909163381275106113451286931993707  # 一个大素数
q = (p-1)//2
g = 5  # p的原根

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16) % p
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16) % p
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16) % q
print('e2:', e2)

s2 = (r - x2 * e2) % q
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

msg: Hello, Ring Signature
pubkeys: [12342984613107202293339795186719190137, 152665891358578884806463863320084678415, 292773863476606314921771127011238797598]
Y: 12342984613107202293339795186719190137152665891358578884806463863320084678415292773863476606314921771127011238797598
e3: 164687035535111599570009344558848081419
random_s3: 257541936627009438398173417538821003870
e1: 145813987555041560589600840265792936193
random_s1: 260171641548194598121706445766479277457
e2: 4755353380164072547438211327749506304
s2: 41908460187659479722945237397485304046
signature process total result:


{'e1': 145813987555041560589600840265792936193,
 's1': 260171641548194598121706445766479277457,
 's2': 41908460187659479722945237397485304046,
 's3': 257541936627009438398173417538821003870}

In [441]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)

79782620466521882912273873893911956468289586534620962686075201655941063208351
92167420401962970013678746745338160460491534602253858691556458062025772926382
69603658134531376683352097715950281523652313907489461048876765979998014119325
False


In [442]:
(pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p

261560713341851367975734800545358598660

In [443]:
pow(g, r, p)

42304195821529907130378650741573395047

In [444]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.backends import default_backend

def generate_dsa_parameters():
    # Generate DSA parameters with 1024-bit p and 160-bit q
    parameters = dsa.generate_parameters(key_size=1024, backend=default_backend())
    
    # Generate private and public keys based on the generated parameters
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()

    # Access the DSA parameters
    p = parameters.parameter_numbers().p
    q = parameters.parameter_numbers().q
    g = parameters.parameter_numbers().g

    print("Generated DSA Parameters:")
    print("p:", p)
    print("q:", q)
    print("g:", g)
    print("\nPrivate Key:", private_key)
    print("Public Key:", public_key)
    return p,q,g

p,q,g = generate_dsa_parameters()

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16) % p
print('e3:', e3)

_, random_s3 = generate_keys(g, q)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16) % p
print('e1:', e1)

_, random_s1 = generate_keys(g, q)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16) % p
print('e2:', e2)

s2 = (r - x2 * e2) % q
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

Generated DSA Parameters:
p: 179267443891248860254939514430752232148189824163768753050332632254953809453335106645892564114948638629252645331084883369448428416374045934317914802991299183017417033922917997102531506329087626193577988481568054151840081651630176903667923157158065751827361336023220831605406164441934473317109346231773825873999
q: 1429392879337943091689125951822528436375009819447
g: 10379232353411704571219882187842266100719486704541527001995635780425419631475915047600726278345894366277043297772970392658225814011435862838929471092329718462461174649602617547781315477656859240264525923586379724461039847993018781212153158644959810877816393531197433255855714772205210581659978815418323070578

Private Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPrivateKey object at 0x0000021D11D673D0>
Public Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPublicKey object at 0x0000021D11D67650>
msg: Hello, Ring Signature
pubkeys: [93902153824935655886390202232510858039897849121455

{'e1': 52870908907941062786778386702178115158233491072327341728383932170884060954611,
 's1': 45592014699932989631523253814191560349023091288,
 's2': 571292168840070697798399476932524854984470702211,
 's3': 202534328525657743077884979180742532376823526675}

In [445]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)

52870908907941062786778386702178115158233491072327341728383932170884060954611
42567490944278793259865458724901112117477923098597010451936208345516100791267
113494521128346651099627761371903136101005313024718177468677652597149363464562
True
